In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence, RunnablePassthrough, RunnableBranch # Core LCEL components

# Load environment variables from a .env file. 🌍
# This ensures your OpenAI API key is loaded securely from your environment.
load_dotenv()

True

In [2]:
# Initialize the ChatOpenAI language model. 🤖
# This model will be used for both generating the report and summarizing it.
model = ChatOpenAI()

# Initialize a StrOutputParser. 📄
# This parser extracts the raw string content from the LLM's response.
parser = StrOutputParser()

In [3]:
# Define the first prompt template for generating a detailed report. 📝
# It takes a 'topic' as input and asks the LLM to write a detailed report.
prompt1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# Define the second prompt template for summarizing text. 📝
# It takes 'text' as input and instructs the LLM to summarize it.
prompt2 = PromptTemplate(
    template='Summarize the following text \n {text}',
    input_variables=['text']
)

In [4]:
# Define the `report_gen_chain`. ⛓️
# This is a simple sequential chain that:
# 1. Takes the initial `topic` input.
# 2. Formats it using `prompt1` to create the report generation prompt.
# 3. Sends the prompt to the `model` to generate the detailed report.
# 4. Uses the `parser` to extract the raw report string.
# The output of this chain will be the detailed report text.
report_gen_chain = prompt1 | model | parser

In [5]:
# Define the conditional branching logic using `RunnableBranch`. 🚦
# `RunnableBranch` allows the workflow to choose a path based on a condition.
# It takes a series of (condition, runnable) pairs, and an optional default runnable.
branch_chain = RunnableBranch(
    # First branch: If the length of the input text (output from report_gen_chain)
    # is greater than 300 words, then execute the summarization chain.
    # `lambda x: len(x.split()) > 300` is the condition function. `x` is the input
    # received by `branch_chain` (which is the detailed report text).
    # `prompt2 | model | parser` is the runnable to execute if the condition is true.
    (lambda x: len(x.split()) > 300, prompt2 | model | parser),

    # Default branch: If the condition above is FALSE (i.e., the report is 300 words or less),
    # then `RunnablePassthrough()` is executed.
    # `RunnablePassthrough()` simply takes its input and passes it through unchanged.
    # So, if the report is short, it won't be summarized; it will be returned as is.
    RunnablePassthrough()
)

In [6]:
# Define the final overall chain. 🔗
# This chain combines `report_gen_chain` and `branch_chain` sequentially.
# 1. `report_gen_chain` runs first, generating the detailed report.
# 2. The *output* of `report_gen_chain` (the detailed report text) becomes the *input* to `branch_chain`.
# 3. `branch_chain` then evaluates its condition:
#    - If the report is long (>300 words), it summarizes it.
#    - If the report is short (<=300 words), it passes the original report through.
final_chain = RunnableSequence(report_gen_chain, branch_chain)

# Invoke the `final_chain` with the initial topic. 🚀
# The input `{'topic':'Russia vs Ukraine'}` starts the entire process.
# The output will either be the summarized report or the original detailed report,
# depending on its length.
print(final_chain.invoke({'topic':'India vs England'}))

The text highlights the longstanding cricket rivalry between India and England, two of the most competitive teams in the world. They have faced each other in numerous Test series, ODIs, and T20 matches, with both sides boasting a strong cricketing history and passionate fan base. The rivalry is not limited to the cricket field, as the two countries have a long history of sporting and cultural exchanges. In recent years, India has had the upper hand in Test cricket, while England has shown success in limited-overs cricket. The matches between the two teams are always fiercely competitive, exciting, and closely contested, with both sides striving for success and supremacy in the cricketing world.
